# Pytorch Neural Network Model for Classification

In [1]:
# Importing modules
import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import classification_report

In [2]:
# Setting up device
device = "cuda" if torch.cuda.is_available() else "cpu"
device= torch.device(device)

# Creating synthetic data for classification
X, y = make_classification( n_samples=100, n_features=4, n_redundant=0,
                            n_informative=2,  n_clusters_per_class=2, n_classes=2)

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [3]:
# Class for converting data to pytorch tensor
class ToTensorData(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

# Converting data to tensor
data = {"train_data": ToTensorData(X_train, y_train), 
        "test_data":ToTensorData(X_test, y_test)}

# Dataloader
dataloader = {"training":DataLoader(dataset=data["train_data"], batch_size=32, shuffle=True),
              "testing":DataLoader(dataset=data["test_data"], batch_size=32)}

In [4]:
# Model
model = nn.Sequential(nn.Linear(4, 4), nn.Sigmoid(), # Hidden layer-1
                      nn.Linear(4, 1), nn.Sigmoid()  # Hidden layer-2
                     ).to(device)

# Model parameters
loss_fn = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [5]:
# Training
for epoch in range(1000):
    for X, y in dataloader["training"]:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y.unsqueeze(-1))
        loss.backward()
        optimizer.step()

In [6]:
# Testing
with torch.no_grad():
    for X, y in dataloader["testing"]:
        y_test = y
        outputs = model(X)
        y_pred = [1 if y >= 0.5 else 0 for y in outputs]
        print(classification_report(y, y_pred))

              precision    recall  f1-score   support

         0.0       0.67      0.89      0.76         9
         1.0       0.88      0.64      0.74        11

    accuracy                           0.75        20
   macro avg       0.77      0.76      0.75        20
weighted avg       0.78      0.75      0.75        20

